Example: Windowed Gaussian Process transition model

In [76]:
# define constants

from sklearn.gaussian_process.kernels import RBF
import numpy as np
from datetime import datetime, timedelta

PRIOR_SCALE = 10
WINDOW_SIZE = 5
TIME_INTERVAL = timedelta(seconds=1)

KERNEL_LENGTH_SCALE = 2
KERNEL_OUT_VAR = 30

NOISE_COVAR = 2
PRIOR_COVAR = 1
num_steps = 30

np.random.seed(0)

In [ ]:
# Define our 2D transition model
from stonesoup.models.transition.linear import CombinedLinearGaussianTransitionModel, \
                                               SlidingWindowGaussianProcess

# Sliding window GP with RBF kernel
class RBFKernelGP(SlidingWindowGaussianProcess):
    def kernel(self, x, y, **kwargs):
        k = KERNEL_OUT_VAR*RBF(KERNEL_LENGTH_SCALE)
        return k(x,y)

transition_model = CombinedLinearGaussianTransitionModel([RBFKernelGP(window_size=WINDOW_SIZE),
                                                          RBFKernelGP(window_size=WINDOW_SIZE)])

In [78]:
# Generate ground truth with SlidingWindowGP

from stonesoup.types.groundtruth import GroundTruthPath, GroundTruthState
from stonesoup.types.state import GaussianState, StateVector

start_time = datetime.now().replace(microsecond=0) 
curr_time = start_time + (WINDOW_SIZE-1) * TIME_INTERVAL  # first prediction at t = 5

# Random prior
prior_state_vector = np.random.rand(WINDOW_SIZE*2)*PRIOR_SCALE
truth = GroundTruthPath([GroundTruthState(prior_state_vector, timestamp=curr_time)])

# Generate path
for k in range(num_steps):
    curr_time += TIME_INTERVAL
    truth.append(GroundTruthState(transition_model.function(truth[-1], noise=True, pred_time=(curr_time - start_time), time_interval=TIME_INTERVAL), timestamp=curr_time))


In [79]:
# Simulate gaussian noise measurements
from stonesoup.types.detection import Detection
from stonesoup.models.measurement.linear import LinearGaussian

# Define measurement model
measurement_model = LinearGaussian(
    ndim_state=WINDOW_SIZE*2,
    mapping=(0, WINDOW_SIZE),
    noise_covar=np.array([[NOISE_COVAR, 0],
                          [0, NOISE_COVAR]])
    )

# Generate measurements
measurements = []
for state in truth:
    measurement = measurement_model.function(state, noise=True)
    measurements.append(Detection(measurement,
                                  timestamp=state.timestamp,
                                  measurement_model=measurement_model))

In [80]:
# Plot ground truth and simulated measurements

from stonesoup.plotter import AnimatedPlotterly

timesteps = [start_time + TIME_INTERVAL * i for i in range(num_steps + WINDOW_SIZE)]

plotter = AnimatedPlotterly(timesteps, tail_length=1)
plotter.plot_ground_truths(truth, [0, WINDOW_SIZE])
plotter.plot_measurements(measurements, [0, WINDOW_SIZE])
plotter.fig

In [81]:
# Initialise predictor and prior for kalman filtering

from stonesoup.predictor.kalman import KalmanPredictor

predictor = KalmanPredictor(transition_model)

prior_state_vector = np.zeros(WINDOW_SIZE*2)
prior = GaussianState(StateVector(prior_state_vector), np.identity(WINDOW_SIZE*2)*PRIOR_COVAR, timestamp=truth[0].timestamp)

In [82]:
# Kalman filtering: Prediction and update steps

from stonesoup.types.hypothesis import SingleHypothesis
from stonesoup.types.track import Track
from stonesoup.updater.kalman import KalmanUpdater

updater = KalmanUpdater(measurement_model)
track = Track()
track.append(prior)

for i in range(1, len(measurements)):
    measurement = measurements[i]
    prediction = predictor.predict(prior, timestamp=measurement.timestamp, pred_time=(measurement.timestamp-start_time))
    hypothesis = SingleHypothesis(prediction, measurement)
    post = updater.update(hypothesis)
    track.append(post)
    prior = track[-1]

# zero division error if prior.timestamp == measurement.timestamp

In [ ]:
# Using Kalman Smoother is equivalent to using last elem in sliding window
# Kalman update step smoothes previous states

In [ ]:
plotter.plot_tracks(track, [0, WINDOW_SIZE], track_label="Sliding window GP", uncertainty=True)
plotter.fig